### Elastic Container Registry (ECR)
##### Check Basic Scanning Version

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ecr = sessions.client('ecr', region_name = region['RegionName'])
            status = ecr.get_account_setting(
                name = 'BASIC_SCAN_TYPE_VERSION'
            )        
            print(' - '+region['RegionName']+' '+status['value'])
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Switch Improved Basic Scanning

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ecr = sessions.client('ecr', region_name = region['RegionName'])
            status = ecr.get_account_setting(
                name = 'BASIC_SCAN_TYPE_VERSION'
            )        
            if status['value'] == 'CLAIR':
                response = ecr.put_account_setting(
                    name='BASIC_SCAN_TYPE_VERSION',
                    value='AWS_NATIVE'
                )
                print(' - '+region['RegionName']+' '+response['value'])
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Get Registry Scanning Configuration

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ecr = sessions.client('ecr', region_name = region['RegionName'])
            status = ecr.get_registry_scanning_configuration()
            print(' - '+region['RegionName']+' '+status['scanningConfiguration']['scanType']+' '+status['scanningConfiguration']['rules'][0]['scanFrequency'])
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Put Registry Scanning Configuration

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ecr = sessions.client('ecr', region_name = region['RegionName'])
            ecr.put_registry_scanning_configuration(
                scanType = 'BASIC',
                rules = [
                    {
                        'scanFrequency': 'SCAN_ON_PUSH',
                        'repositoryFilters': [
                            {
                                'filter': '*',
                                'filterType': 'WILDCARD'
                            }
                        ]
                    }
                ]
            )
            print(' - '+region['RegionName']+' SCAN_ON_PUSH')
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Scan Container Images

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            repo_count = 0
            img_count = 0
            ecr = sessions.client('ecr', region_name = region['RegionName'])
            paginator = ecr.get_paginator('describe_repositories')
            response_iterator = paginator.paginate()
            ### Repository Loop ###
            for page in response_iterator:
                for repository in page['repositories']:
                    repo_count += 1
                    paginator2 = ecr.get_paginator('list_images')
                    response_iterator2 = paginator2.paginate(
                        repositoryName = repository['repositoryName'],
                    )
                    ### Image Loop ###
                    for page2 in response_iterator2:
                        for imageid in page2['imageIds']:
                            img_count += 1
                            try:
                                ecr.start_image_scan(
                                    registryId = repository['registryId'],
                                    repositoryName = repository['repositoryName'],
                                    imageId = {
                                        'imageDigest': imageid['imageDigest'],
                                        'imageTag': imageid['imageTag']
                                    }
                                )
                            except:
                                if display_denied == True:
                                    print('Scan Quota Exceeded: '+ imageid['imageDigest'])
                                pass
            print(' - '+region['RegionName']+' Repositories: '+str(repo_count)+' Images: '+str(img_count))
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Container Vulnerbility Summary

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ecr = sessions.client('ecr', region_name = region['RegionName'])
            paginator = ecr.get_paginator('describe_repositories')
            response_iterator = paginator.paginate()
            ### Repository Loop ###
            for page in response_iterator:
                for repository in page['repositories']:
                    paginator2 = ecr.get_paginator('list_images')
                    response_iterator2 = paginator2.paginate(
                        repositoryName = repository['repositoryName'],
                    )
                    ### Image Loop ###
                    for page2 in response_iterator2:
                        for imageid in page2['imageIds']:
                            findings = ecr.describe_image_scan_findings(
                                registryId = repository['registryId'],
                                repositoryName = repository['repositoryName'],
                                imageId = {
                                    'imageDigest': imageid['imageDigest'],
                                    'imageTag': imageid['imageTag']
                                }
                            )
                            if findings['imageScanStatus']['status'] != 'FAILED':
                                if len(findings['imageScanFindings']['findingSeverityCounts']) > 0:
                                    print(' - '+region['RegionName']+' '+findings['repositoryName']+' '+findings['imageId']['imageTag']+' '+findings['imageScanStatus']['status']+' '+str(findings['imageScanFindings']['findingSeverityCounts']))
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass